In [1]:
from sklearn import ensemble as ens
from sklearn import datasets
from sklearn import model_selection as cv
from sklearn import metrics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
df = pd.read_csv('cars_data.csv')
df.head()

,Make,Model,Type,Origin,DriveTrain,MSRP,Invoice,EngineSize,Cylinders,Horsepower,MPG_City,MPG_Highway,Weight,Wheelbase,Length
0,Acura,MDX,SUV,Asia,All,"$36,945","$33,337",3.5,6.0,265,17,23,4451,106,189
1,Acura,RSX Type S 2dr,Sedan,Asia,Front,"$23,820","$21,761",2.0,4.0,200,24,31,2778,101,172
2,Acura,TSX 4dr,Sedan,Asia,Front,"$26,990","$24,647",2.4,4.0,200,22,29,3230,105,183
3,Acura,TL 4dr,Sedan,Asia,Front,"$33,195","$30,299",3.2,6.0,270,20,28,3575,108,186
4,Acura,3.5 RL 4dr,Sedan,Asia,Front,"$43,755","$39,014",3.5,6.0,225,18,24,3880,115,197


In [7]:
del df['Invoice']
df.dropna(inplace=True)

y = df.pop('MSRP').str.replace('$', '').str.replace(',', '').astype(int)
catCols = ['Make','Model','Type','Origin', 'DriveTrain']

xCat = pd.get_dummies(df[catCols])
df.drop(catCols, axis=1, inplace=True)
X = pd.concat([df, xCat], axis=1)

xtrain, xtest, ytrain, ytest = cv.train_test_split(X, y, test_size=0.3, random_state=1)

/home/jaidevd/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  after removing the cwd from sys.path.


In [8]:
bag = ens.BaggingRegressor(random_state=1)
rfc = ens.RandomForestRegressor(random_state=1)
gb = ens.GradientBoostingRegressor(random_state=1)
ada = ens.AdaBoostRegressor(random_state=1)

for clf in [bag, rfc, gb, ada]:
    clf.fit(xtrain, ytrain)

In [9]:
bag.score(xtest, ytest)

0.7949163237522345

In [10]:
rfc.score(xtest, ytest)

0.8369440882741959

In [11]:
gb.score(xtest, ytest)

0.8270485242007507

In [12]:
ada.score(xtest, ytest)

0.711903287964231

In [14]:
grid = {'n_estimators': [100, 500, 1000], 'learning_rate': [0.5, 1, 2]}

gcv = cv.GridSearchCV(ada, grid, cv=4, n_jobs=-1)
gcv.fit(xtrain, ytrain)

GridSearchCV(cv=4, estimator=AdaBoostRegressor(random_state=1), n_jobs=-1,
             param_grid={'learning_rate': [0.5, 1, 2],
                         'n_estimators': [100, 500, 1000]})

In [15]:
gcv.score(xtest, ytest)

0.7130800769863186

In [16]:
gcv.best_params_

{'learning_rate': 1, 'n_estimators': 1000}